In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json
import requests
from datetime import date
import datetime
import helper



def lambda_handler(event, context):
    sql_query = pd.read_sql('SELECT * FROM city', con)
    cities = dict(zip(sql_query.city_id, sql_query.city))
    wiki_df = pd.DataFrame(sql_query)
    icao_list = pd.read_sql('SELECT icao FROM airport', con)["icao"].unique().tolist()
    
    helper.get_weather(cities)
    helper.get_arrivals(icao_list)

    return {}

In [ ]:
# functions stored in helper.py

# Weather Function
def get_weather(cities): 

    time = []
    temp = []
    weather = []
    rain = []
    city = []
    city_id = []
    weather_id = []
    
    API_key = "API_key"
    tomorrow = pd.to_datetime((datetime.date.today() + datetime.timedelta(days=1)).strftime("%Y-%m-%d"))
    
    schema="schema"
    host="AWS_host"
    dotenv.load_dotenv() # the api key is in a .env file in the same directory
    password = os.getenv('sql_password')
    user="admin"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    
    for key,c in cities.items():
        json = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={c}&appid={API_key}&units=metric").json()
        
        for i in range(len(json["list"])):
            city.append(c)
            city_id.append(key)
            time.append(json["list"][i]["dt_txt"])
            temp.append(json["list"][i]["main"]["temp"])
            print(key + json["list"][i]["dt_txt"])
            weather_id.append(key + json["list"][i]["dt_txt"])
            weather.append(json["list"][i]["weather"][0]["main"])
            try:
                rain.append(json["list"][i]["rain"]["3h"])
            except:
                rain.append(0)
    df = pd.DataFrame({"w_id":weather_id,
                "city_id":city_id,
                #"city":city,
                "forecast_time":time, 
                "temperature":temp,
                 "weather":weather, 
                 "rain":rain})
    df["forecast_time"] = pd.to_datetime(df["forecast_time"])
    df = df[df["forecast_time"].dt.date==tomorrow.date()]
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    df.to_sql('weather', 
             if_exists='append', 
             con=con, 
             index=False)
    return(df)
    
 
    
# ARRIVALS function
def get_arrivals(icao_list):
    schema="sql_gans"
    host="wbs-project5-db.cqsgae76sx7r.us-east-2.rds.amazonaws.com"
    user="admin"
    password="Datascience!23"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    arrival_key = []
    list = []
    tomorrow = (datetime.date.today() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    
    for i in icao_list:

        #original
        #url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/ICAO/2023-04-04T20:00/2023-04-05T08:00"
        
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{i}/{tomorrow}T08:00/{tomorrow}T20:00"
    
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

        headers = {
        "X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
    
        response = requests.get(url, headers=headers, params=querystring)
        print(i, response)
        try:
            df = pd.DataFrame(pd.json_normalize(response.json()["arrivals"]))
            df = df[["number", "movement.airport.icao", "movement.airport.name","movement.actualTimeLocal", "movement.terminal", "airline.name"]]
            df.rename(columns={"movement.airport.name":"departure_airport", "movement.airport.icao":"departure_icao", "movement.actualTimeLocal":"arrival_time", "movement.terminal":"arrival_terminal", "airline.name":"airline"}, inplace=True)  
            df["arrival_icao"] = i
            list.append(df)
        except:
            pass
    arrival_df = pd.concat(list)
    arrival_df.to_sql('arrival', 
              if_exists='append', 
              con=con, 
              index=False)
    return (arrival_df)